In [39]:
import numpy as np
import threading as thr
import time
import random as rand
import pandas as pd

In [40]:
df = pd.read_csv("london_weather.csv")

In [41]:
# load the data
df = pd.read_csv('london_weather.csv')

# 1) drop any rows with missing values
df_clean = df.dropna()

# 2) remove the 'date' column if it exists
if 'date' in df_clean.columns:
    df_clean = df_clean.drop(columns=['date'])

df = df_clean.copy()

In [42]:
class lstmModel:
  def __init__(self, alpha):
    self.weights = np.zeros(6)            # 6 “weights”
    self.grad    = np.zeros_like(self.weights)
    self.alpha   = alpha

  def zero_grad(self):
    self.grad.fill(0.)

  def train(self, chunk):
    # 1) pretend our “model” is y_pred = X @ w
    #    split chunk into X (6 features) and y (1 target)
    X = chunk.iloc[:, :6].values           # shape (batch,6)
    y = chunk.iloc[:, 6].values            # shape (batch,)

    # 2) forward → MSE loss
    y_pred = X.dot(self.weights)           # (batch,)
    error  = y_pred - y                    # (batch,)
    loss   = np.mean(error**2)             # scalar, not used further

    # 3) backward → ∂(MSE)/∂w = (2/batch) * Xᵀ·error
    self.zero_grad()
    self.grad = (2.0 / X.shape[0]) * X.T.dot(error)  # shape (6,)

    # simulate a bit of compute time
    time.sleep(5)

    # 4) return that gradient vector
    return self.grad

  def updateWeights(self):
    self.weights -= self.alpha * self.grad

In [43]:
# Hyper params
N = 3
grads = [None]*N
models = [None]*N
epochs = 5

In [44]:
chunks = np.array_split(df, N)
chunks[i]

/home/saatvikk/.local/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
9257,6.0,4.5,177.0,16.9,10.8,6.8,0.0,99420.0,0.0
9258,7.0,6.1,203.0,12.7,12.2,7.4,3.4,100240.0,0.0
9259,8.0,0.0,75.0,15.6,11.1,9.5,1.4,100310.0,0.0
9260,8.0,0.9,108.0,19.5,12.8,10.1,2.2,100800.0,0.0
9261,6.0,7.7,229.0,15.7,16.0,12.6,0.0,101330.0,0.0
...,...,...,...,...,...,...,...,...,...
14970,8.0,0.0,12.0,9.6,7.5,6.6,0.0,102880.0,0.0
14971,8.0,0.0,12.0,9.9,8.1,6.6,0.0,103630.0,0.0
14972,6.0,2.2,29.0,11.5,8.7,7.4,0.0,103360.0,0.0
14973,0.0,3.6,37.0,9.3,6.5,1.5,0.0,102970.0,0.0


In [45]:

compBarrier = thr.Barrier(N+1)
updateBarrier = thr.Barrier(N+1)

def aggregate(grads):
  return sum(grads) / len(grads)

mutex = thr.Lock()
def addGrad(i, grad):
  print("THREAD", i, "IS CURRENTLY WAITING FOR THE `grads` MUTEX TO BE ACQUIRED...")
  mutex.acquire()
  try: grads[i] = grad
  finally: 
    mutex.release()
    print("THREAD", i, "HAS RELEASED THE `grads` MUTEX...")

def worker(threadNum, chunk):
  LSTMi = models[threadNum]
  for epoch in range(epochs):
    print("================================")
    grad = LSTMi.train(chunk)
    addGrad(threadNum, grad)
    print("THREAD:", threadNum, ", EPOCH:", epoch, "GRAD:", grad)
    compBarrier.wait()
    print("THREAD", threadNum, "is waiting for compBarrier. EPOCH:", epoch)
    updateBarrier.wait()
    print("THREAD", threadNum, "is waiting for updateBarrier. EPOCH:", epoch)
    print("================================")

M = df.shape[0]
chunks = np.array_split(df, N)
threads = []
for i, chunk in enumerate(chunks):
  models[i] = lstmModel(0.01)
  t = thr.Thread(target=worker, args=(i, chunk), daemon=True)
  t.start()
  print("Started thread", i, " using chunk", i, ".")
  threads.append(t)
  
for epoch in range(epochs):
  print("================================")
  compBarrier.wait()           # wait for all workers to finish computing
  newGrad = aggregate(grads)
  print("[main] NEW GRAD:", newGrad)
  for m in models:
    m.grad = newGrad        # or call m.updateWeights() if you stash grad there
    m.updateWeights()
  updateBarrier.wait()         # let workers continue to next epoch

for t in threads:
    t.join()


/home/saatvikk/.local/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Started thread 0  using chunk 0 .
Started thread 1  using chunk 1 .
Started thread 2  using chunk 2 .
THREAD 1 IS CURRENTLY WAITING FOR THE `grads` MUTEX TO BE ACQUIRED...
THREAD 1 HAS RELEASED THE `grads` MUTEX...
THREAD: 1 , EPOCH: 0 GRAD: [ -22.30325098   -7.72842653 -284.92436064  -48.28209796  -38.59735154
  -27.6534287 ]
THREAD 2 IS CURRENTLY WAITING FOR THE `grads` MUTEX TO BE ACQUIRED...
THREAD 2 HAS RELEASED THE `grads` MUTEX...
THREAD: 2 , EPOCH: 0 GRAD: [ -20.65509319   -7.02138708 -257.08040746  -47.06417859  -36.63600347
  -25.45453403]
THREAD 0 IS CURRENTLY WAITING FOR THE `grads` MUTEX TO BE ACQUIRED...
THREAD 0 HAS RELEASED THE `grads` MUTEX...
THREAD: 0 , EPOCH: 0 GRAD: [ -20.79562297   -7.66872806 -293.92472373  -44.41227736  -34.7768104
  -23.48357963]
THREAD 0 is waiting for compBarrier. EPOCH: 0
[main] NEW GRAD: [ -21.25132238   -7.47284722 -278.64316394  -46.58618464  -36.67005514
  -25.53051412]
THREAD 1 is waiting for compBarrier. EPOCH: 0
THREAD 2 is waiting fo